In [1]:
from Launch import  Scheduler
import unittest
from datetime import datetime, timedelta, timezone



In [2]:
# როდესაც ჯობიისთის გადაცემული გაშვების თარიღი

In [ ]:

class TestScheduler(unittest.TestCase):
    # 
    # def test_scheduler_initialization(self):
    #     """Test the initialization of the Scheduler class."""
    #     scheduler = Scheduler(threading=False, start_date="2024-12-25 10:00", time_zone=0)
    #     self.assertEqual(scheduler.scheduler_startdate.strftime('%Y-%m-%d %H:%M'), "2024-12-25 10:00")
    #     self.assertEqual(scheduler.scheduler_timezone, timezone(timedelta(hours=0)))
    # 
    # def test_invalid_start_date(self):
    #     """Test that invalid start_date raises a ValueError."""
    #     with self.assertRaises(ValueError):
    #         Scheduler(start_date="invalid-date", time_zone=0)
    # 
    # 
    # def test_job_creation(self):
    #     """Test creating a job within the scheduler."""
    #     timezonesc = 5
    #     timezonejob = 0
    #     local_timezone = datetime.now().astimezone().tzinfo
    # 
    # 
    #     start_datesc = "2024-12-25 10:00"
    #     startdatejob = "2024-12-25 11:00"
    #     scheduler = Scheduler(start_date=start_datesc, time_zone=timezonesc)
    #     job_to_run = scheduler.job(start_date=startdatejob, time_zone=timezonejob)
    #     
    #     if timezonejob:
    #         if local_timezone != timezonejob:
    #             start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
    #             comparison = start_date.astimezone(local_timezone)
    #             self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
    #         else:
    #             self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)
    #     if not timezonejob and timezonejob:
    #         if timezonejob != local_timezone:
    #             start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
    #             comparison = start_date.astimezone(local_timezone)
    #             self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
    #         else:
    #             self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)



    # def test_job_invalid_timezone(self):
    #     """Test that invalid timezone in job raises a ValueError."""
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
    #     with self.assertRaises(ValueError):
    #         scheduler.job(start_date="2024-12-25 11:00", time_zone="invalid")
    # 
    # def test_job_repeat(self):
    #     """Test that a job can be set to repeat a specified number of times."""
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
    #     job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).repeat(5)
    #     self.assertEqual(job.job['repeats'], 5)
    # 
    # def test_job_interval(self):
    #     """Test setting intervals for a job."""
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
    #     job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).minute(2)
    #     self.assertEqual(job.job['interval'], 2)
    #     self.assertEqual(job.job['unit'], 'minute')

    def test_job_end_date(self):
        """Test setting an end date for a job."""
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        end_date = "2024-12-26 10:00"
        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).until(end_date)
        self.assertEqual(job.job['end_date'].strftime('%Y-%m-%d %H:%M'), end_date)

#     def test_run_all_with_threading(self):
#         """Test running all jobs with threading enabled."""
#         scheduler = Scheduler(threading=True, start_date="2024-12-25 10:00", time_zone=0)
# 
#         def dummy_task():
#             print("Task executed")
# 
#         job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).do(dummy_task, name="TestJob")
#         scheduler.run_all()
#         self.assertIsNotNone(job.job['func'])
# 
#     def test_run_all_without_threading(self):
#         """Test running all jobs without threading."""
#         scheduler = Scheduler(threading=False, start_date="2024-12-25 10:00", time_zone=0)
# 
#         def dummy_task():
#             print("Task executed")
# 
#         job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).do(dummy_task, name="TestJob")
#         scheduler.run_all()
#         self.assertIsNotNone(job.job['func'])
# 
#     def test_calculate_next_run(self):
#         """Test calculation of the next run time for a job."""
#         scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
#         job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).minute(1)
#         now = datetime(2024, 12, 25, 11, 0, 0)
#         next_run = job.calculate_next_run(now)
#         expected_next_run = now + timedelta(minutes=1)
#         self.assertEqual(next_run, expected_next_run)
# 
# if __name__ == "__main__":
#     unittest.main(argv=[''], exit=False, verbosity=2)
# 
# 


test_calculate_next_run (__main__.TestScheduler.test_calculate_next_run)
Test calculation of the next run time for a job. ... ok
test_invalid_start_date (__main__.TestScheduler.test_invalid_start_date)
Test that invalid start_date raises a ValueError. ... ok
test_job_creation (__main__.TestScheduler.test_job_creation)
Test creating a job within the scheduler. ... ok
test_job_end_date (__main__.TestScheduler.test_job_end_date)
Test setting an end date for a job. ... FAIL
test_job_interval (__main__.TestScheduler.test_job_interval)
Test setting intervals for a job. ... ok
test_job_invalid_timezone (__main__.TestScheduler.test_job_invalid_timezone)
Test that invalid timezone in job raises a ValueError. ... ok
test_job_repeat (__main__.TestScheduler.test_job_repeat)
Test that a job can be set to repeat a specified number of times. ... ok
test_run_all_with_threading (__main__.TestScheduler.test_run_all_with_threading)
Test running all jobs with threading enabled. ... 

this is scheduler startdate and timezone inside scheduler class:
 UTC
2024-12-25 14:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

calc next run: 2024-12-25 11:01:00
this is scheduler startdate and timezone inside scheduler class:
 UTC+05:00
2024-12-25 09:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

this is scheduler startdate and timezone inside scheduler class:
 UTC
2024-12-25 14:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

This is the job's timezone and the converted end date in 'until':
UTC
2024-12-26 14:00:00+04:00

this is job timezone abd end date in untill:
 UTC
 2024-12-26 06:00:00+00:00

this is scheduler startdate and timezone inside scheduler class:
 UTC
2024-12-25 14:00:00+04:00

this is job timezone UTC
this is job startdate and timezo